# Сглаживание для шестизвенного робота

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import animation
import numpy as np
from IPython.display import HTML
%matplotlib notebook

In [ ]:
from kinematics import Vector, Quaternion, Transform
import graphics

Возьмем шестизвенного робота:

In [ ]:
def irb_chain(q, l):
    base = Transform.identity()
    column = base + Transform(
        Vector(0, 0, l[0]),
        Quaternion.from_angle_axis(q[0], Vector(0, 0, 1))
    )
    shoulder = column + Transform(
        Vector(l[1], 0, 0),
        Quaternion.from_angle_axis(q[1], Vector(0, -1, 0))
    )
    elbow = shoulder + Transform(
        Vector(0, 0, l[2]),
        Quaternion.from_angle_axis(q[2], Vector(0, 1, 0))
    )
    wrist = elbow + Transform(
        Vector(l[3], 0, 0),
        Quaternion.from_angle_axis(q[3], Vector(1, 0, 0)) *
        Quaternion.from_angle_axis(q[4], Vector(0, 1, 0))
    )
    flange = wrist + Transform(
        Vector(l[4], 0, 0),
        Quaternion.from_angle_axis(q[5], Vector(1, 0, 0)) *
        Quaternion.from_angle_axis(np.pi / 2, Vector(0, 1, 0))
    )
    return [
        base,
        column,
        shoulder,
        elbow,
        wrist,
        flange
    ]

In [ ]:
def wrap_from_to(value, s, e):
    r = e - s
    return value - (r * np.floor((value - s) / r))

def irb_ik(target, l, i=[1, 1, -1]):
    wrist = target + Vector(0, 0, -l[4]) +  Vector(0, 0, -l[0])
    projection = Vector(wrist.x, wrist.y, 0)
    q0 = Vector(0, 1, 0).angle_to(projection, Vector(0, 0, 1)) - np.pi / 2 * i[0] + np.pi
    d = ((projection.magnitude() - i[0] * l[1]) ** 2 + wrist.z ** 2) ** 0.5
    q2 = -i[1] * np.arccos(
        (l[2] **  2 + l[3] ** 2 - d ** 2) /\
        (2 * l[2] * l[3])
    ) + np.pi / 2
    triangle_angle = np.arcsin(
        l[3] * i[0] * np.sin(q2 - np.pi / 2) / d
    )
    lift_angle = np.arctan2(
        wrist.z,
        (projection.magnitude() - i[0] * l[1])
    )
    q1 = -i[0] * (np.pi / 2 + triangle_angle - lift_angle)
    ori = Quaternion.from_angle_axis(q0, Vector(0, 0, 1)) *\
        Quaternion.from_angle_axis(q1, Vector(0, -1, 0)) *\
        Quaternion.from_angle_axis(q2, Vector(0, 1, 0))
    ez = ori * Vector(1, 0, 0)
    ey = ori * Vector(0, 1, 0)
    tz = target.rotation * Vector(0, 0, 1)
    ty = target.rotation * Vector(0, 1, 0)
    wy = ez.cross(tz)
    q3 = ey.angle_to(wy, ez) + np.pi / 2 - np.pi / 2 * i[2]
    q4 = ez.angle_to(tz, wy) * i[2]
    q5 = wy.angle_to(ty, tz) + np.pi / 2 -np.pi / 2 * i[2]
    return (
        wrap_from_to(q0, -np.pi, np.pi),
        wrap_from_to(q1, -np.pi, np.pi),
        wrap_from_to(q2, -np.pi, np.pi),
        wrap_from_to(q3, -np.pi, np.pi),
        wrap_from_to(q4, -np.pi, np.pi),
        wrap_from_to(q5, -np.pi, np.pi)
    )

In [ ]:
irb_l = [352.0, 70.0, 350.0, 380.0, 65.0]

In [ ]:
s = Transform(
    Vector(200, 400, 600),
    Quaternion.from_angle_axis(np.pi / 2, Vector(-1, 0, 0))
)
i = Transform(
    Vector(400, 100, 900),
    Quaternion.from_angle_axis(np.pi / 4, Vector(0, 1, 0))
)
e = Transform(
    Vector(200, -300, 800),
    Quaternion.from_angle_axis(np.pi / 2, Vector(0, 1, 0))
)
blend = 0.1

## Линейное-линейное

In [ ]:
def lin(l, start, end, t, total):
    target = Transform.lerp(
        start,
        end,
        t / total
    )
    return irb_ik(target, l)
    

def lin_lin(l, start, inter, end, t, total):
    progress = t / total
    if progress < 0.5:
        return lin(l, start, inter, progress, 0.5)
    return lin(l, inter, end, progress - 0.5, 0.5)

def lin_lin_smooth(l, start, inter, end, t, total, blend=0.2):
    progress = t / total
    if np.abs(progress - 0.5) < blend:
        progress = (progress - 0.5 + blend) / 2 / blend
        a = Transform.lerp(start, inter, 1.0 - 2 * blend)
        b = inter
        c = Transform.lerp(inter, end, 2 * blend)
        trg = Transform.lerp(
            Transform.lerp(a, b, progress),
            Transform.lerp(b, c, progress),
            progress
        )
        return irb_ik(trg, l)
    return lin_lin(l, start, inter, end, t, total)

In [ ]:
(x, y, z) = graphics.chain_to_points(
    irb_chain([0, 0, 0, 0, 0, 0], irb_l)
)
fig, ax = graphics.figure(1000)
lines, = ax.plot(x, y, z, color="#000000")
graphics.axis(ax, s, 100)
graphics.axis(ax, i, 100)
graphics.axis(ax, e, 100)

total = 100

def animate(frame):
    q = lin_lin_smooth(irb_l, s, i, e, frame, total, blend)
    chain = irb_chain(q, irb_l)
    (x, y, z) = graphics.chain_to_points(chain)
    lines.set_data_3d(x, y, z)


animate(0)
fps = 25
ani = animation.FuncAnimation(
    fig,
    animate,
    frames=total,
    interval=1000.0/fps
)

In [ ]:
v_lin_lin_smooth = np.vectorize(lin_lin_smooth, excluded={0, 1, 2, 3, 5, 6})
total = 100
step = 0.1
t = np.arange(0, total, step)
q = v_lin_lin_smooth(irb_l, s, i, e, t, total, blend)
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(2, 2, 1)
ax.plot(t, q[0], label="$q_0$")
ax.plot(t, q[1], label="$q_1$")
ax.plot(t, q[2], label="$q_2$")
ax.plot(t, q[3], label="$q_3$")
ax.plot(t, q[4], label="$q_4$")
ax.plot(t, q[5], label="$q_5$")
ax.legend()

a3 = fig.add_subplot(2, 2, 2, projection="3d")
(x, y, z) = graphics.chain_to_points(irb_chain(q, irb_l))
a3.plot(x[-1], y[-1], z[-1])

w = np.diff(q)
aw = fig.add_subplot(2, 2, 3)
aw.plot(t[:-1], w[0] / step, label="$\omega_0$")
aw.plot(t[:-1], w[1] / step, label="$\omega_1$")
aw.plot(t[:-1], w[2] / step, label="$\omega_2$")
aw.plot(t[:-1], w[3] / step, label="$\omega_3$")
aw.plot(t[:-1], w[4] / step, label="$\omega_4$")
aw.plot(t[:-1], w[5] / step, label="$\omega_5$")

eps = np.diff(w)
ae = fig.add_subplot(2, 2, 4)
ae.plot(t[:-2], eps[0] / step ** 2, label="$\epsilon_0$")
ae.plot(t[:-2], eps[1] / step ** 2, label="$\epsilon_1$")
ae.plot(t[:-2], eps[2] / step ** 2, label="$\epsilon_2$")
ae.plot(t[:-2], eps[3] / step ** 2, label="$\epsilon_3$")
ae.plot(t[:-2], eps[4] / step ** 2, label="$\epsilon_4$")
ae.plot(t[:-2], eps[5] / step ** 2, label="$\epsilon_5$")
ae.legend()

aw.legend()
fig.show()

## Переброска-переброска

In [ ]:
def ptp(l, start, end, t, total):
    s = irb_ik(start, l)
    e = irb_ik(end, l)
    q = (
        s[0] + (e[0] - s[0]) * t / total,
        s[1] + (e[1] - s[1]) * t / total,
        s[2] + (e[2] - s[2]) * t / total,
        s[3] + (e[3] - s[3]) * t / total,
        s[4] + (e[4] - s[4]) * t / total,
        s[5] + (e[5] - s[5]) * t / total
    )
    return q

def ptp_ptp(l, start, inter, end, t, total):
    progress = t / total
    if progress < 0.5:
        return ptp(l, start, inter, progress, 0.5)
    return ptp(l, inter, end, progress - 0.5, 0.5)

def ptp_ptp_smooth(l, start, inter, end, t, total, blend=0.2):
    progress = t / total
    if np.abs(progress - 0.5) < blend:
        progress = (progress - 0.5 + blend) / 2 / blend
        a = ptp(l, start, inter, 1.0 - 2 * blend, 1)
        b = irb_ik(inter, l)
        c = ptp(l, inter, end, 2 * blend, 1)
        return (
            a[0] * (1 - progress) ** 2 + b[0] * 2 * progress * (1 - progress) + c[0] * progress ** 2,
            a[1] * (1 - progress) ** 2 + b[1] * 2 * progress * (1 - progress) + c[1] * progress ** 2,
            a[2] * (1 - progress) ** 2 + b[2] * 2 * progress * (1 - progress) + c[2] * progress ** 2,
            a[3] * (1 - progress) ** 2 + b[3] * 2 * progress * (1 - progress) + c[3] * progress ** 2,
            a[4] * (1 - progress) ** 2 + b[4] * 2 * progress * (1 - progress) + c[4] * progress ** 2,
            a[5] * (1 - progress) ** 2 + b[5] * 2 * progress * (1 - progress) + c[5] * progress ** 2,
        )
    return ptp_ptp(l, start, inter, end, t, total)

In [ ]:
(x, y, z) = graphics.chain_to_points(
    irb_chain([0, 0, 0, 0, 0, 0], irb_l)
)
fig, ax = graphics.figure(1000)
lines, = ax.plot(x, y, z, color="#000000")
graphics.axis(ax, s, 100)
graphics.axis(ax, i, 100)
graphics.axis(ax, e, 100)

total = 100

def animate(frame):
    q = ptp_ptp_smooth(irb_l, s, i, e, frame, total, blend)
    chain = irb_chain(q, irb_l)
    (x, y, z) = graphics.chain_to_points(chain)
    lines.set_data_3d(x, y, z)


animate(0)
fps = 25
ani = animation.FuncAnimation(
    fig,
    animate,
    frames=total,
    interval=1000.0/fps
)

In [ ]:
v_ptp_ptp_smooth = np.vectorize(ptp_ptp_smooth, excluded={0, 1, 2, 3, 5, 6})
total = 100
step = 0.1
t = np.arange(0, total, step)
q = v_ptp_ptp_smooth(irb_l, s, i, e, t, total, blend)
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(2, 2, 1)
ax.plot(t, q[0], label="$q_0$")
ax.plot(t, q[1], label="$q_1$")
ax.plot(t, q[2], label="$q_2$")
ax.plot(t, q[3], label="$q_3$")
ax.plot(t, q[4], label="$q_4$")
ax.plot(t, q[5], label="$q_5$")
ax.legend()

a3 = fig.add_subplot(2, 2, 2, projection="3d")
(x, y, z) = graphics.chain_to_points(irb_chain(q, irb_l))
a3.plot(x[-1], y[-1], z[-1])

w = np.diff(q)
aw = fig.add_subplot(2, 2, 3)
aw.plot(t[:-1], w[0] / step, label="$\omega_0$")
aw.plot(t[:-1], w[1] / step, label="$\omega_1$")
aw.plot(t[:-1], w[2] / step, label="$\omega_2$")
aw.plot(t[:-1], w[3] / step, label="$\omega_3$")
aw.plot(t[:-1], w[4] / step, label="$\omega_4$")
aw.plot(t[:-1], w[5] / step, label="$\omega_5$")

eps = np.diff(w)
ae = fig.add_subplot(2, 2, 4)
ae.plot(t[:-2], eps[0] / step ** 2, label="$\epsilon_0$")
ae.plot(t[:-2], eps[1] / step ** 2, label="$\epsilon_1$")
ae.plot(t[:-2], eps[2] / step ** 2, label="$\epsilon_2$")
ae.plot(t[:-2], eps[3] / step ** 2, label="$\epsilon_3$")
ae.plot(t[:-2], eps[4] / step ** 2, label="$\epsilon_4$")
ae.plot(t[:-2], eps[5] / step ** 2, label="$\epsilon_5$")
ae.legend()

aw.legend()
fig.show()

## Линейное-переброска

Наивная реализация

In [ ]:
def lin_ptp(l, start, inter, end, t, total):
    progress = t / total
    if progress < 0.5:
        return lin(l, start, inter, progress, 0.5)
    return ptp(l, inter, end, progress - 0.5, 0.5)

def lin_ptp_smooth(l, start, inter, end, t, total, blend=0.2):
    progress = t / total
    if np.abs(progress - 0.5) < blend:
        progress = (progress - 0.5 + blend) / 2 / blend
        
        l_blend_i = inter
        q_blend_i = irb_ik(inter, l)
        
        l_blend_s = Transform.lerp(start, inter, 1 - 2 * blend)
        q_blend_s = irb_ik(l_blend_s, l)
        
        q_blend_e = ptp(l, inter, end, 2 * blend, 1)
        l_blend_e = irb_chain(q_blend_e, l)[-1]
        
        l_blend = irb_ik(
            Transform.lerp(
                Transform.lerp(l_blend_s, l_blend_i, progress),
                Transform.lerp(l_blend_i, l_blend_e, progress),
                progress
            ),
            l
        )
        q_blend = (
            q_blend_s[0] * (1 - progress) ** 2 + q_blend_i[0] * 2 * (1 - progress) * progress + q_blend_e[0] * progress ** 2,
            q_blend_s[1] * (1 - progress) ** 2 + q_blend_i[1] * 2 * (1 - progress) * progress + q_blend_e[1] * progress ** 2,
            q_blend_s[2] * (1 - progress) ** 2 + q_blend_i[2] * 2 * (1 - progress) * progress + q_blend_e[2] * progress ** 2,
            q_blend_s[3] * (1 - progress) ** 2 + q_blend_i[3] * 2 * (1 - progress) * progress + q_blend_e[3] * progress ** 2,
            q_blend_s[4] * (1 - progress) ** 2 + q_blend_i[4] * 2 * (1 - progress) * progress + q_blend_e[4] * progress ** 2,
            q_blend_s[5] * (1 - progress) ** 2 + q_blend_i[5] * 2 * (1 - progress) * progress + q_blend_e[5] * progress ** 2,
        )
        return (
            l_blend[0] + (q_blend[0] - l_blend[0]) * progress,
            l_blend[1] + (q_blend[1] - l_blend[1]) * progress,
            l_blend[2] + (q_blend[2] - l_blend[2]) * progress,
            l_blend[3] + (q_blend[3] - l_blend[3]) * progress,
            l_blend[4] + (q_blend[4] - l_blend[4]) * progress,
            l_blend[5] + (q_blend[5] - l_blend[5]) * progress,
        )
    return lin_ptp(l, start, inter, end, t, total)

In [ ]:
(x, y, z) = graphics.chain_to_points(
    irb_chain([0, 0, 0, 0, 0, 0], irb_l)
)
fig, ax = graphics.figure(1000)
lines, = ax.plot(x, y, z, color="#000000")
graphics.axis(ax, s, 100)
graphics.axis(ax, i, 100)
graphics.axis(ax, e, 100)

total = 100

def animate(frame):
    q = lin_ptp_smooth(irb_l, s, i, e, frame, total, blend)
    chain = irb_chain(q, irb_l)
    (x, y, z) = graphics.chain_to_points(chain)
    lines.set_data_3d(x, y, z)


animate(50)
fps = 25
ani = animation.FuncAnimation(
    fig,
    animate,
    frames=total,
    interval=1000.0/fps
)

In [ ]:
v_lin_ptp_smooth = np.vectorize(lin_ptp_smooth, excluded={0, 1, 2, 3, 5, 6})
total = 100
step = 0.1
t = np.arange(0, total, step)
q = v_lin_ptp_smooth(irb_l, s, i, e, t, total, blend)
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(2, 2, 1)
ax.plot(t, q[0], label="$q_0$")
ax.plot(t, q[1], label="$q_1$")
ax.plot(t, q[2], label="$q_2$")
ax.plot(t, q[3], label="$q_3$")
ax.plot(t, q[4], label="$q_4$")
ax.plot(t, q[5], label="$q_5$")
ax.legend()

a3 = fig.add_subplot(2, 2, 2, projection="3d")
(x, y, z) = graphics.chain_to_points(irb_chain(q, irb_l))
a3.plot(x[-1], y[-1], z[-1])

w = np.diff(q)
aw = fig.add_subplot(2, 2, 3)
aw.plot(t[:-1], w[0] / step, label="$\omega_0$")
aw.plot(t[:-1], w[1] / step, label="$\omega_1$")
aw.plot(t[:-1], w[2] / step, label="$\omega_2$")
aw.plot(t[:-1], w[3] / step, label="$\omega_3$")
aw.plot(t[:-1], w[4] / step, label="$\omega_4$")
aw.plot(t[:-1], w[5] / step, label="$\omega_5$")

eps = np.diff(w)
ae = fig.add_subplot(2, 2, 4)
ae.plot(t[:-2], eps[0] / step ** 2, label="$\epsilon_0$")
ae.plot(t[:-2], eps[1] / step ** 2, label="$\epsilon_1$")
ae.plot(t[:-2], eps[2] / step ** 2, label="$\epsilon_2$")
ae.plot(t[:-2], eps[3] / step ** 2, label="$\epsilon_3$")
ae.plot(t[:-2], eps[4] / step ** 2, label="$\epsilon_4$")
ae.plot(t[:-2], eps[5] / step ** 2, label="$\epsilon_5$")
ae.legend()

aw.legend()
fig.show()